# 과제 1: Movie Agent — 완전한 에이전트 루프

## 기능
- 실제 Movie API를 호출하는 3개의 도구 (`get_popular_movies`, `get_movie_details`, `get_similar_movies`)
- OpenAI `tools` 파라미터를 사용한 함수 호출
- 에이전트 루프: LLM이 최종 답변을 줄 때까지 도구 호출 → 결과 반환 반복
- 메모리를 활용한 멀티턴 대화

## API
- 기본 URL: `https://nomad-movies.nomadcoders.workers.dev`
- `/movies` — 인기 영화 목록
- `/movies/:id` — 영화 상세 정보
- `/movies/:id/similar` — 유사 영화 목록

In [1]:
import openai
import dotenv
import requests
import json

dotenv.load_dotenv()

client = openai.OpenAI()

BASE_URL = "https://nomad-movies.nomadcoders.workers.dev"

## 1. API 호출 함수 정의

In [2]:
def get_popular_movies():
    """인기 영화 목록을 가져옵니다."""
    response = requests.get(f"{BASE_URL}/movies")
    return response.json()


def get_movie_details(movie_id: int):
    """특정 영화의 상세 정보를 가져옵니다."""
    response = requests.get(f"{BASE_URL}/movies/{movie_id}")
    return response.json()


def get_similar_movies(movie_id: int):
    """특정 영화와 유사한 영화 목록을 가져옵니다."""
    response = requests.get(f"{BASE_URL}/movies/{movie_id}/similar")
    return response.json()

## 2. OpenAI Tools 정의

In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_popular_movies",
            "description": "현재 인기 있는 영화 목록을 가져옵니다.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_movie_details",
            "description": "특정 영화의 상세 정보(줄거리, 평점, 개봉일, 장르 등)를 가져옵니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "movie_id": {
                        "type": "integer",
                        "description": "영화의 고유 ID",
                    }
                },
                "required": ["movie_id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_similar_movies",
            "description": "특정 영화와 비슷한 영화 목록을 가져옵니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "movie_id": {
                        "type": "integer",
                        "description": "영화의 고유 ID",
                    }
                },
                "required": ["movie_id"],
            },
        },
    },
]

## 3. 에이전트 루프

In [5]:
# 함수 이름 → 실제 함수 매핑
available_functions = {
    "get_popular_movies": get_popular_movies,
    "get_movie_details": get_movie_details,
    "get_similar_movies": get_similar_movies,
}

SYSTEM_PROMPT = """당신은 영화 전문가 에이전트입니다.

사용자의 영화 관련 질문에 답하기 위해 다음 도구를 활용할 수 있습니다:
- get_popular_movies: 현재 인기 영화 목록 조회
- get_movie_details: 특정 영화의 상세 정보 조회 (ID 필요)
- get_similar_movies: 특정 영화와 유사한 영화 조회 (ID 필요)

규칙:
- 답변은 항상 한국어로 해주세요.
- 영화 정보를 제공할 때 영화 ID도 함께 알려주세요.
- 대화 맥락을 기억하고, 이전에 언급된 영화의 ID를 활용하세요.
- 사용자가 특정 영화에 대해 더 알고 싶다고 하면 get_movie_details를 사용하세요.
- 비슷한 영화를 요청하면 get_similar_movies를 사용하세요."""

# 대화 기록 (멀티턴 메모리)
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
]


def run_agent(user_message: str):
    """에이전트 루프: 도구 호출이 없을 때까지 반복합니다."""
    print(f"👤 User: {user_message}\n")
    messages.append({"role": "user", "content": user_message})

    while True:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools,
        )

        choice = response.choices[0]
        message = choice.message

        # assistant 메시지를 대화 기록에 추가
        messages.append(message)

        # 도구 호출이 없으면 최종 답변 → 루프 종료
        if not message.tool_calls:
            print(f"🤖 Agent: {message.content}\n")
            print(f"--- 대화 기록: {len(messages)}개 메시지 ---")
            return

        # 도구 호출 처리
        for tool_call in message.tool_calls:
            fn_name = tool_call.function.name
            fn_args = json.loads(tool_call.function.arguments)

            print(f"🔧 도구 호출: {fn_name}({fn_args})")

            # 실제 API 호출
            fn = available_functions[fn_name]
            result = fn(**fn_args)

            # 도구 결과를 대화 기록에 추가
            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": json.dumps(result, ensure_ascii=False),
                }
            )

        print()  # 가독성을 위한 빈 줄

## 4. 테스트

In [6]:
# 테스트 1: 인기 영화 조회
run_agent("지금 인기 있는 영화 알려줘")

👤 User: 지금 인기 있는 영화 알려줘

🔧 도구 호출: get_popular_movies({})

🤖 Agent: 현재 인기 있는 영화 목록은 다음과 같습니다:

1. **Shelter**
   - ID: 1290821
   - 줄거리: 한 남자가 외딴 섬에서 자신이 겪은 폭풍으로부터 젊은 소녀를 구하고, 과거의 적들로부터 그녀를 보호하기 위해 세상과 다시 맞서게 됩니다.
   - 개봉일: 2026-01-28
   - 평점: 6.8
   - ![포스터](https://image.tmdb.org/t/p/w780/buPFnHZ3xQy6vZEHxbHgL1Pc6CR.jpg)

2. **The Bluff**
   - ID: 799882
   - 줄거리: 평화로운 섬에서의 삶이 복수심에 불타는 과거의 선장이 돌아오면서 흔들리게 되고, 그녀는 가족을 구하기 위해 과거의 그녀를 직면하게 됩니다.
   - 개봉일: 2026-02-17
   - 평점: 6.0
   - ![포스터](https://image.tmdb.org/t/p/w780/sojEzvfxR2DBcDSJyAisX8TWjov.jpg)

3. **Mercy**
   - ID: 1236153
   - 줄거리: 가까운 미래, 한 형사가 아내를 살해한 혐의로 재판을 받고 있으며, 자신의 결백을 입증하기 위해 첨단 AI 판사와의 싸움을 벌입니다.
   - 개봉일: 2026-01-20
   - 평점: 7.1
   - ![포스터](https://image.tmdb.org/t/p/w780/pyok1kZJCfyuFapYXzHcy7BLlQa.jpg)

4. **Hellfire**
   - ID: 1088434
   - 줄거리: 신비로운 과거를 가진 방랑자가 작은 마을에 도착하고, 주민들이 범죄 보스의 지배를 받고 있는 것을 보고 그들을 돕기로 결심합니다.
   - 개봉일: 2026-02-05
   - 평점: 7.1
   - ![포스터](https://image.tmdb.org/t/p/w780/tQti9QTf13MfzNpXguijg

In [7]:
# 테스트 2: 특정 영화 상세 정보 (이전 대화에서 나온 영화 ID 활용)
run_agent("첫 번째 영화에 대해 더 자세히 알려줘")

👤 User: 첫 번째 영화에 대해 더 자세히 알려줘

🔧 도구 호출: get_movie_details({'movie_id': 1290821})

🤖 Agent: **Shelter**에 대한 자세한 정보는 다음과 같습니다:

- **원제**: Shelter
- **개봉일**: 2026년 1월 28일
- **장르**: 액션, 범죄, 스릴러
- **러닝타임**: 107분
- **감독**: 정보 없음
- **출연진**: 정보 없음

**줄거리**: 한 남자가 자신이 선택한 고립의 삶을 살며 외딴 섬에서 지내던 중, 폭풍으로부터 젊은 소녀를 구하게 되고, 이는 그의 과거와 관련된 적들로부터 그녀를 지키기 위해 그가 세상과 다시 맞서게 만드는 사건의 연쇄를 시작하게 됩니다.

**예산**: 50,000,000 달러  
**수익**: 26,786,795 달러  
**아이맥스 ID**: tt32357218

**태그라인**: "Her safety. His mission."

**제작사**:
- Black Bear Pictures
- Punch Palace Productions
- CineMachine
- Stampede Ventures

**상세 페이지**: [영화 공식 홈페이지](https://blackbearpictures.com/film-and-tv/shelter)

**언어**: 영어

**평점**: 6.8 (152표)

![포스터](https://image.tmdb.org/t/p/w780/buPFnHZ3xQy6vZEHxbHgL1Pc6CR.jpg)

더 궁금한 점이나 다른 영화에 대해 알고 싶으시면 말씀해 주세요!

--- 대화 기록: 9개 메시지 ---


In [8]:
# 테스트 3: 유사 영화 추천
run_agent("그 영화랑 비슷한 영화 추천해줘")

👤 User: 그 영화랑 비슷한 영화 추천해줘

🔧 도구 호출: get_similar_movies({'movie_id': 1290821})

🤖 Agent: **Shelter**와 비슷한 영화를 다음과 같이 추천드립니다:

1. **Roseville**
   - ID: 291163
   - 줄거리: 1985년 초가을, 경치 좋은 발칸 산맥의 외딴 산장에서 발생한 충격적이고 신비로운 사건 이후 경찰이 그 결과를 보게 됩니다. 젊은 커플이 휴가로 찾아가면서 벌어지는 불가사의한 사건들이 그들의 정신을 시험하게 됩니다.
   - 개봉일: 2013년 12월 20일
   - 평점: 5.6
   - ![포스터](https://image.tmdb.org/t/p/w780/mmQnPUUdw6a1D9kkmDPRDQK08zj.jpg)

2. **Esther's Choice**
   - ID: 736573
   - 줄거리: 영화 작곡가 에스더가 남자친구와 함께 살기 시작하면서 그가 예전 여자친구의 물건을 아직도 갖고 있는 것을 발견하게 되고, 그로 인해 여러 감정적인 사건이 벌어집니다.
   - 개봉일: 2020년 10월 15일
   - 평점: 평가 없음
   - ![포스터](https://image.tmdb.org/t/p/w780/t7jJv2tE2w06zJ8JAzNVeJKZyBH.jpg)

3. **Backtrack**
   - ID: 332704
   - 줄거리: 걱정되는 심리 치료사인 피터가 악몽과 괴로운 환영에 시달리고, 그의 모든 환자가 공유하는 horrifying 비밀을 발견하면서 그가 20년 전에 도망쳤던 고향으로 돌아가게 되는 이야기를 다룹니다.
   - 개봉일: 2015년 7월 25일
   - 평점: 5.8
   - ![포스터](https://image.tmdb.org/t/p/w780/8L4jifugHPASkrssmOUVdQxn3WN.jpg)

4. **Vesper**
   - ID: 567321
   - 줄거리: 마르지가 자신의 남편으로부터 스스로를 보호하기 위해 

In [9]:
# 테스트 4: 메모리 확인 — 이전 대화 맥락 기억하는지 확인
run_agent("아까 내가 처음에 뭐 물어봤었지?")

👤 User: 아까 내가 처음에 뭐 물어봤었지?

🤖 Agent: 처음에 인기 있는 영화에 대해 물어보셨습니다. 제가 현재 인기 있는 영화 목록을 제공해 드렸습니다. 다시 궁금한 점이 있으면 말씀해 주세요!

--- 대화 기록: 15개 메시지 ---
